# Object Detection

In [1]:
from expt_utils import *

## Background Model Initialization

In [2]:
frames_dir = f'{DS_DIR}/test/images'

frames_files = sorted([
    os.path.join(frames_dir, file_name)
    for file_name in os.listdir(frames_dir)
    if file_name.endswith('.jpg') and file_name.split('_')[0] == '00062'
])

num_frames = len(frames_files)

base_frame = plt.imread(frames_files[-1])
base_frame = scale_img(base_frame, 0.6)
base_frame_gray = cv.cvtColor(base_frame, cv.COLOR_RGB2GRAY).astype(np.float32)


def _render(frame_idx):
    img = cv.imread(frames_files[frame_idx])
    img = scale_img(img, 0.6)
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY).astype(np.float32)
    img_diff = cv.absdiff(img_gray, base_frame_gray).astype(np.uint8)
    fg_mask = cv.threshold(img_diff, 30, 255, cv.THRESH_BINARY)[1]
    cv.imshow('Frame', img)
    cv.imshow('Differential Frame', img_diff)
    cv.imshow('Foreground Mask', fg_mask)


_render(frame_idx=0)
cv.createTrackbar('Frame Idx', 'Frame', 0, num_frames-1, _render)

destroy_when_esc()

## Adaptive Background Subtraction with Gaussian Mixture Models (GMM)

In [3]:
bg_subtractor = cv.createBackgroundSubtractorMOG2()


def _render(frame_idx):
    img = cv.imread(frames_files[frame_idx])
    img = scale_img(img, 0.6)
    fg_mask = bg_subtractor.apply(img)
    cv.imshow('Frame', img)
    cv.imshow('Foreground Mask', fg_mask)


_render(frame_idx=0)
cv.createTrackbar('Frame', 'Frame', 0, num_frames-1, _render)

destroy_when_esc()